In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1613053627772_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, \
case when (contains(r.observations, 'LC') )  then 1 else 0 end as LC \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US','en_CA','en_GB') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-01-01' \
"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head()
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... LC
0  322203888  ...  0
1  322090421  ...  0
2  322202219  ...  0
3  322068282  ...  0
4  322198180  ...  0

[5 rows x 5 columns]

In [6]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8347146 entries, 0 to 8347145
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UGC_ID           int64 
 1   NAME             object
 2   REVIEW_HEADLINE  object
 3   REVIEW_COMMENTS  object
 4   LC               int64 
 5   ALL_TEXT         object
dtypes: int64(2), object(4)
memory usage: 382.1+ MB

In [8]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [9]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                       NAME  \
0  322203888  Discount Ramps              
1  322090421  EverlyWell Seller Ratings   
2  322202219  Lions Den                   
3  322068282  Finish Line                 
4  322198180  Finish Line                 

                               REVIEW_HEADLINE  \
0  Perfect solution                              
1  Great!                                        
2  I would buy again an would recommend          
3  I would definitely recommend copping these!   
4  Love the shoes for my son                     

                                                                                                                                                                                                                                                                                                                                                                                                                   REVIEW_COMMENTS  \
0  Very easy to in

In [17]:
#Remove mentions of people cheating on their diets, feelings like I'm cheating
reviews['regex_cheat_diet'] = reviews['ALL_TEXT'].str.replace(r'((FEELS? LIKE (I\'M|IM|I AM)|FEELS? LIKE (YOUR|YOU\'RE|YOURE|YOU)) CHEAT)|(CHEAT MEAL)', '', case=False)



#\b before and after sued so don't match suede or issued
#reviews['regex_LC'] = reviews['ALL_TEXT'].str.contains(r'(\bSCAM\b|FALSE ADVERTIS(ING|EMENT)|FRAUD|BAIT (AND|&) SWITCH|PRICE GOUG(E|ING)|DECEPTIVE\b|DECEIT|FAKE (?!TAN|EYELASH|TREE|LASH)|\bPHONY|CHEAT(ED)?\b|\bRIP(PED)?[ -]?OFF|BETTER BUSINESS BUREAU|\bBBB\b|THIEF|THEIF|\bSUED\b|LAWSUIT)', case=False)==True
reviews['regex_LC'] = reviews['regex_cheat_diet'].str.contains(r'(\bSCAM\b|FALSE ADVERTIS(ING|EMENT)|FRAUD|BAIT (AND|&) SWITCH|PRICE GOUG(E|ING)|DECEPTIVE\b|DECEIT|\bPHONY|CHEAT(ED)?\b|\bRIP(PED)?[ -]?OFF|BETTER BUSINESS BUREAU|\bBBB\b|THIEF|THEIF|\bSUED\b|LAWSUIT)', case=False)==True


#reviews['cheat']=reviews['ALL_TEXT'].str.contains(r'(CHEAT)', case=False)==True

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:2: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [47]:
reviews['fake'] = reviews['regex_cheat_diet'].str.contains(r'(FAKE\s(?!TAN|EYELASH|TREE|LASH|PLANT|NAIL|(([A-Z]+\s)?(SMELL|FLAVOR))))', case=False)==True


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [48]:
test = reviews[reviews['fake']==True]
test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UGC_ID               4355
NAME                 4355
REVIEW_HEADLINE      4355
REVIEW_COMMENTS      4355
LC                   4355
ALL_TEXT             4355
regex_cheat_diet     4355
regex_cheat_diet2    4355
regex_LC             4355
model_LC             4355
fraud                4355
fake                 4355
dtype: int64

In [52]:
test2= reviews.loc[(reviews['fake']==True) & (reviews['LC']==1)]
test2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UGC_ID               718
NAME                 718
REVIEW_HEADLINE      718
REVIEW_COMMENTS      718
LC                   718
ALL_TEXT             718
regex_cheat_diet     718
regex_cheat_diet2    718
regex_LC             718
model_LC             718
fraud                718
fake                 718
dtype: int64

In [63]:
reviews['fake_LC'] = reviews['regex_cheat_diet'].str.contains(r'(FAKE (PRODUCT|NEWS|REVIEW|PROMISE|SALE|ADVERTISE|ITEM)|IT\'?S FAKE)', case=False)==True


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [64]:
test3= reviews.loc[(reviews['fake']==True) & (reviews['LC']==1) & (reviews['fake_LC']==False)]
test3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UGC_ID               576
NAME                 576
REVIEW_HEADLINE      576
REVIEW_COMMENTS      576
LC                   576
ALL_TEXT             576
regex_cheat_diet     576
regex_cheat_diet2    576
regex_LC             576
model_LC             576
fraud                576
fake                 576
fake_LC              576
dtype: int64

In [62]:
test3.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            UGC_ID                               NAME  \
92       322204816  Fingerhut                           
5375     318422249  Fingerhut                           
15679    321903556  Perfume.com                         
26124    314222632  Nutrisystem                         
31154    325926329  Hammacher Schlemmer                 
38079    318094116  Balsam Hill                         
51335    318907470  Cost Plus World Market              
60305    328628388  Tesco Grocery                       
64316    304799152  FragranceX                          
65523    265403727  Tempur Pedic                        
68362    321117304  HOKA ONE ONE                        
68705    321119372  HOKA ONE ONE                        
118540   268679547  Tiger Medical                       
119459   317757579  Fingerhut                           
119574   327705249  EverlyWell                          
121656   308269046  Agency                              
132505   317798853  Ulta       

In [18]:
reviews['model_LC'] = 0

reviews.loc[(reviews['regex_LC']==True ) , 'model_LC'] = 1


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
FP = reviews[(reviews['LC'] == 0) & (reviews['model_LC'] == 1)]
FN = reviews[(reviews['LC'] == 1) & (reviews['model_LC'] == 0)]
TP = reviews[(reviews['LC'] == 1) & (reviews['model_LC'] == 1)]
TN = reviews[(reviews['LC'] == 0) & (reviews['model_LC'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#TEST WITHOUT FAKE IN REGEX
#TONS OF MENTIONS OF FAKE SMELL, FAKE OIL, ETC ETC NOT RELATING TO LEGAL COMPLAINTS
#9494 with fake, 5205 without

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4797

In [21]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10475

In [22]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9365

In [23]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8322509

In [24]:
FP.head(200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           UGC_ID                                        NAME  \
2636    293472635  Experian CC Marketplace                      
3113    265391979  Ulta                                         
4547    275905547  SmartEtailing                                
7673    284250730  Life Extension Seller Ratings                
7904    295674813  Discount Tire                                
8662    318128250  Kate Aspen                                   
10103   275906686  Consumer Reports                             
12034   321933862  Artful Home                                  
13613   264510220  Instant Pot US - TERMINATED                  
14775   270272630  Tempur Pedic                                 
14835   270187198  Citigroup Inc.                               
14847   270220317  Citigroup Inc.                               
15612   321881969  MidwayUSA                                    
18626   322331314  Books-A-Million                              
19253   314265536  Vans I

In [59]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEWER_NICKNAME", StringType(), True)\
,StructField("BASE_URL", StringType(), True)\
,StructField("URL", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("client_URL_1", StringType(), True)\
,StructField("client_URL_2", StringType(), True)\
,StructField("regex_URL", StringType(), True)\
,StructField("regex_extract", StringType(), True)\
,StructField("client_URL_mention_1", StringType(), True)\
,StructField("client_URL_mention_2", StringType(), True)\
,StructField("bomb_com", StringType(), True)\
,StructField("model_URL", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data is already a DataFrame
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 672, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame



In [69]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","URL_FP_test1").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_BATCH_TRACKING_TABLE")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_FULL_TRACKING_TABLE")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.URL_FN_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.CC_FP_TEST7")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…